In [38]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json
import folium


ModuleNotFoundError: No module named 'folium'

In [39]:
df  = pd.read_pickle("data/data.pkl")
df['Date'] = df['Votation'].map(lambda x : x.split(' ')[0])
df['Votation'] = df['Votation'].map(lambda x : ' '.join(x.split(' ')[1:]))
df

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
0,Aeugst am Albis,Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,29.11.1998
1,Aeugst am Albis,Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,14.06.2015
2,Aeugst am Albis,Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,25.09.2016
3,Aeugst am Albis,Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse,03.03.1991
4,Aeugst am Albis,Réforme de l'imposition des entreprises III,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,12.02.2017
5,Aeugst am Albis,Recherche sur l'être humain,1262.0,588.0,46.6,579.0,476.0,103.0,82.2,Affoltern,Zürich,Suisse,07.03.2010
6,Aeugst am Albis,Initiative pour l'internement des délinquants ...,1135.0,590.0,52.0,578.0,266.0,312.0,46.0,Affoltern,Zürich,Suisse,08.02.2004
7,Aeugst am Albis,"L'or à l'AVS, aux cantons et à la Fondation",1113.0,610.0,54.8,604.0,325.0,267.0,54.9,Affoltern,Zürich,Suisse,22.09.2002
8,Aeugst am Albis,Contre les abus dans le droit d'asile,1117.0,662.0,59.3,648.0,282.0,366.0,43.5,Affoltern,Zürich,Suisse,24.11.2002
9,Aeugst am Albis,Suppression de la vivisection,683.0,363.0,53.1,358.0,135.0,223.0,37.7,Affoltern,Zürich,Suisse,01.12.1985


In [40]:
xls = pd.ExcelFile('data/Recommandations des partis.xls')
nb_sheets = len(xls.sheet_names)
recommand = pd.DataFrame()

for i in range ((2017-1981 + 1)) :
#for i in range (6,7) :
 
    x = xls.parse(i)
    x.columns = range(len(x.columns))
    
    base_nb = x[x.iloc[:,0] == 'Parti 1)'].index[0]
    

    x=x.dropna(axis = 1 , how = 'all')

    recommand_inter = pd.concat([x.iloc[base_nb:base_nb+1],x.iloc[base_nb+2:base_nb+3],x.dropna()]).transpose()
    
    recommand_inter.iloc[0,2:] =  recommand_inter.iloc[0,2:].map(lambda x : x.split(' ')[0])    
    
    
    recommand_inter.columns = range(len(recommand_inter.columns))
    recommand_inter.iloc[:,0] = recommand_inter.iloc[:,0]+str(2017-i)
    
    recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map( lambda x : x[0].upper() + x[1:] ,\
                                                              na_action = 'ignore')
   

    propositions = x[x.iloc[:,0].str.contains('No ').fillna(False)]
    propositions.columns = range (len( propositions.columns))
    
    if  (propositions.iloc[:,1].isnull()).all() :
        if ( 2017-i == 1997) :
            propositions.iloc[:,1] = propositions[0].map ( lambda x : x.split(' ')[1])
            propositions.iloc[:,0] = propositions[0].map ( lambda x : x.split(' ')[0].rstrip())
        else :
                
            propositions.iloc[:,1] = propositions[0].map ( lambda x : x.split(':')[1][1:])
            propositions.iloc[:,0] = propositions[0].map ( lambda x : x.split(':')[0].rstrip())
    
    
    dico_no_propos = propositions.dropna(1).set_index(0).to_dict()
    if ( dico_no_propos.get(1) == None ) :
        
        recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map(dico_no_propos)
    else :
        recommand_inter.iloc[:,1] = recommand_inter.iloc[:,1].map(dico_no_propos[1])
    recommand_columns = ['Date','Votation'] + list(recommand_inter.iloc[0][2:])
    recommand_inter.columns = recommand_columns
    recommand_inter['Date'] = recommand_inter['Date'].fillna(method = 'ffill')
    recommand_inter=recommand_inter.transpose()
    
    recommand_inter=recommand_inter.groupby(recommand_inter.index).first()
    recommand = recommand.join( recommand_inter , how  = 'outer' , lsuffix='l', rsuffix='r')

    
recommand = recommand.transpose()  

recommand.index = range(len(recommand.index))
recommand = recommand[~recommand.loc[:,'Date'].str.contains('Parti 1').fillna(False)]
recommand.index = range(len(recommand.index))
recommand = recommand[~recommand['Votation'].isnull()]
#recommand = recommand[recommand_columns[1:]]


month_to_int = {'janvier' : '01' , 'février':'02', 'févirer':'02', 'mars':'03' , 'avril':'04' , 'mai':'05' , 'juin':'06','juillet':'07',\
              'aout':'08','septembre':'09','octobre':'10','novembre':'11','décembre':'12'}

def good_format_month (x) :
    if '.' in x :
        split_x = x.split('.')
        return split_x[0].zfill(2) + '.' + split_x[1].zfill(2) + '.' + split_x[2][:4]
    else :
        return str(x.split(' ')[0]).zfill(2) + '.' +  month_to_int[str(x.split(' ')[1]) ] + '.' + x[-4:] 

recommand.loc[:,'Date'] = recommand.loc[:,'Date'].map(lambda x : good_format_month(x))

recommand = recommand.loc[:,recommand.columns.drop_duplicates() ]
parties = list(recommand.columns.drop_duplicates())
parties.remove('Date')
parties.remove('Votation')

recommand = recommand.loc[:,['Date','Votation'] + parties]
recommand

C:\python3\lib\site-packages\pandas\core\indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,POCH,PRD,PS,PSL,PST,PVL,Parti,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,oui,NaN,NaN,oui,NaN,NaN,non,NaN
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,non,NaN,NaN,oui,NaN,NaN,oui,NaN
2,12.02.2017,Réforme de l'imposition des entreprises III,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,non,NaN,NaN,oui,NaN,NaN,oui,NaN
3,21.05.2017,Loi sur l'énergie,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,oui,NaN,NaN,oui,NaN,NaN,non,NaN
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,NaN,NaN,NaN,NaN,non,NaN,oui,non,...,NaN,NaN,non,NaN,NaN,non,NaN,NaN,oui,NaN
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,NaN,NaN,NaN,NaN,non,NaN,non,non,...,NaN,NaN,non,NaN,NaN,non,NaN,NaN,oui,NaN
9,28.02.2016,Initiative contre la spéculation sur les denré...,NaN,NaN,NaN,NaN,non,NaN,non,oui,...,NaN,NaN,oui,NaN,NaN,non,NaN,NaN,non,NaN
10,28.02.2016,Réfection du tunnel routier du Gothard,NaN,NaN,NaN,NaN,oui,NaN,oui,non,...,NaN,NaN,non,NaN,NaN,non,NaN,NaN,oui,NaN
11,05.06.2016,Initiative populaire «En faveur du service pub...,NaN,NaN,NaN,NaN,non,NaN,non,non,...,NaN,NaN,non,NaN,NaN,non,NaN,NaN,non,NaN
12,05.06.2016,Initiative populaire «Pour un revenu de base i...,NaN,NaN,NaN,NaN,non,NaN,non,oui,...,NaN,NaN,non,NaN,NaN,non,NaN,NaN,non,NaN


In [41]:
recommand.columns.drop_duplicates()

Index(['Date', 'Votation', 'AdI', 'DS', 'Lega', 'MCR', 'PBD', 'PCS', 'PDC',
       'PES', 'PEV', 'PLR', 'PLS', 'POCH', 'PRD', 'PS', 'PSL', 'PST', 'PVL',
       'Parti', 'Rep.', 'UDC', 'UDF'],
      dtype='object')

In [42]:
def translate_choice (x) :
    if (x == 1 or str(x) == 'oui') :
        return 1
    elif ( x == 2 or str(x) == 'non'):
        return 0
    else :
        return -1

good_recommand = recommand.copy() 
good_recommand.loc[:,parties] = good_recommand.loc[:,parties].applymap(lambda x: translate_choice (x) )
good_recommand

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,POCH,PRD,PS,PSL,PST,PVL,Parti,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,-1,-1,-1,0,1,-1,1,1,...,-1,-1,1,-1,-1,1,-1,-1,0,-1
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,-1,-1,-1,1,1,-1,1,0,...,-1,-1,0,-1,-1,1,-1,-1,1,-1
2,12.02.2017,Réforme de l'imposition des entreprises III,-1,-1,-1,1,1,-1,1,0,...,-1,-1,0,-1,-1,1,-1,-1,1,-1
3,21.05.2017,Loi sur l'énergie,-1,-1,-1,0,1,-1,1,1,...,-1,-1,1,-1,-1,1,-1,-1,0,-1
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,-1,-1,-1,-1,0,-1,1,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
9,28.02.2016,Initiative contre la spéculation sur les denré...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,1,-1,-1,0,-1,-1,0,-1
10,28.02.2016,Réfection du tunnel routier du Gothard,-1,-1,-1,-1,1,-1,1,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
11,05.06.2016,Initiative populaire «En faveur du service pub...,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,0,-1,-1,0,-1,-1,0,-1
12,05.06.2016,Initiative populaire «Pour un revenu de base i...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,0,-1,-1,0,-1,-1,0,-1


In [43]:
good_recommand[good_recommand['Votation'].isnull()]

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,POCH,PRD,PS,PSL,PST,PVL,Parti,Rep.,UDC,UDF


In [44]:
dico_recom_vote = {}

for date1, group1 in df[df['Commune'] == 'Aeugst am Albis'].groupby('Date'):
    for date2, group2 in good_recommand.groupby('Date'):
        if (date1 == date2) :
            vot1 = list(group1['Votation']) 
            vot2 = list(group2['Votation'])
            while (len(vot1) > 0 and len(vot2) > 0) :
                max_v = -1
                max_match = [0,0]
                for elem1  in vot1 :
                    for elem2 in  vot2 :
                        current_v = SequenceMatcher(None,elem1,elem2).ratio()
                        if(current_v > max_v) :
                            max_v = current_v
                            max_match=[elem1,elem2]
                dico_recom_vote[max_match[0]] = max_match[1]
                dico_recom_vote[max_match[1]] = max_match[0]
                vot1.remove(max_match[0])
                vot2.remove(max_match[1])
            
            
dico_recom_vote

{"10e révision de l'AVS": "Loi fédérale sur l'assurance-vieillesse et survivants",
 "11e révision de l'AVS": "Loi fédérale sur l'assurance-vieillesse et survivants (11e révision de l'AVS)",
 "40 places d'armes ça suffit": "Initiative populaire «40 places d'armes ça suffit! - L'armée doit aussi se soumettre à la législation sur la protection de l'environnement»",
 "5e révision de l'AI": "Loi fédérale sur l'assurance-invalidité",
 '6 semaines de vacances pour tous': 'Initiative populaire «6 semaines de vacances pour tous»',
 "Abaissement de l'âge de la retraite": "Initiative populaire «visant à abaisser à 62 ans pour les hommes et à 60 ans pour les femmes l'âge donnant droit à la rente AVS»",
 "Abandon de l'énergie atomique": "Initiative populaire «pour un abandon progressif de l'énergie atomique»",
 'Abolition de la vivisection': "Initiative populaire «pour l'abolition des expériences sur animaux»",
 'Abrogation du service militaire obligatoire': "Initiative populaire «Oui à I'abrogatio

In [45]:
recommand_to_join = good_recommand.copy()
recommand_to_join['Votation'] = recommand_to_join['Votation'].map(dico_recom_vote)
recommand_to_join

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,POCH,PRD,PS,PSL,PST,PVL,Parti,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,-1,-1,-1,0,1,-1,1,1,...,-1,-1,1,-1,-1,1,-1,-1,0,-1
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,-1,-1,-1,1,1,-1,1,0,...,-1,-1,0,-1,-1,1,-1,-1,1,-1
2,12.02.2017,Réforme de l'imposition des entreprises III,-1,-1,-1,1,1,-1,1,0,...,-1,-1,0,-1,-1,1,-1,-1,1,-1
3,21.05.2017,Loi sur l'énergie,-1,-1,-1,0,1,-1,1,1,...,-1,-1,1,-1,-1,1,-1,-1,0,-1
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,-1,-1,-1,-1,0,-1,1,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
9,28.02.2016,Initiative contre la spéculation sur les denré...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,1,-1,-1,0,-1,-1,0,-1
10,28.02.2016,Réfection du tunnel routier du Gothard,-1,-1,-1,-1,1,-1,1,0,...,-1,-1,0,-1,-1,0,-1,-1,1,-1
11,05.06.2016,Initiative «En faveur du service public»,-1,-1,-1,-1,0,-1,0,0,...,-1,-1,0,-1,-1,0,-1,-1,0,-1
12,05.06.2016,Initiative «Pour un revenu de base incondition...,-1,-1,-1,-1,0,-1,0,1,...,-1,-1,0,-1,-1,0,-1,-1,0,-1


In [46]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))
commune = [x['name']  for x in geo_json_data['features']]

to_map = df.merge(recommand_to_join.loc[:,['Votation']+parties] , on = 'Votation')
to_map = to_map[to_map['Commune'].isin(commune)]
to_map

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,...,POCH,PRD,PS,PSL,PST,PVL,Parti,Rep.,UDC,UDF
0,Aeugst am Albis,Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
1,Affoltern am Albis,Initiative Droleg,5729.0,2286.0,39.9,2236.0,678.0,1558.0,30.3,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
2,Bonstetten,Initiative Droleg,2596.0,1063.0,40.9,1045.0,358.0,687.0,34.3,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
3,Hausen am Albis,Initiative Droleg,2081.0,807.0,38.8,792.0,281.0,511.0,35.5,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
4,Hedingen,Initiative Droleg,1858.0,810.0,43.6,791.0,246.0,545.0,31.1,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
5,Kappel am Albis,Initiative Droleg,612.0,269.0,44.0,264.0,56.0,208.0,21.2,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
6,Knonau,Initiative Droleg,926.0,479.0,51.7,468.0,145.0,323.0,31.0,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
7,Maschwanden,Initiative Droleg,366.0,182.0,49.7,179.0,43.0,136.0,24.0,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
8,Mettmenstetten,Initiative Droleg,2419.0,1100.0,45.5,1081.0,357.0,724.0,33.0,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0
9,Obfelden,Initiative Droleg,2722.0,1023.0,37.6,1002.0,311.0,691.0,31.0,Affoltern,...,-1,0,1,0,1,-1,-1,-1,0,0


In [47]:



for parti in parties :
    current_to_map = to_map.loc[:,['Commune','Oui en %' , parti]]
   
    current_to_map=current_to_map[current_to_map[parti]!=-1]
    current_to_map['Agreement'] = current_to_map [['Oui en %' , parti]].apply ( lambda x : x['Oui en %'] if x[parti] == 1 else 100- x['Oui en %'], axis=1 )
    
    current_to_map=current_to_map.groupby('Commune' , as_index  = False).mean()
    
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                                    data = current_to_map, \
                                    columns = ['Commune', 'Agreement'], \
                                    key_on = 'feature.name', \
                                    fill_color = 'YlGnBu', \
                                    fill_opacity = 0.7, \
                                    line_opacity = 0.2, \
                                    legend_name = 'Agreement in % with ' + partie)
    
    map1.save('map '+partie+'.html')
    print(map1)

NameError: name 'folium' is not defined